<a href="https://colab.research.google.com/github/neiltheblue/loial/blob/main/Loial_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The Loial pacakge needs to be available, as package or from the git repo.

In [2]:
!pip install -i https://test.pypi.org/simple/ loial

Looking in indexes: https://test.pypi.org/simple/


# Simple Python builder example

The current Loial project supports two buildedrs. One is the python builder that is used mainly to exercise the framework, and the CC compiler that is used to integrate compiles libraries. So far this has only been tested using Linux.

This first exmaple shows how a python method can be replaced by an alternative implemention.

In [3]:
def grow(x):
  return x + x

assert grow(3) == 6

The alternative implementation. Here the content of the build decorator is used instead of the python method body.

In [4]:
from loial import build

@build('return x ** 2')
def grow(x):
  return x + x

assert grow(3) == 9

This alternative behaviour can be turned off.

In [5]:
from loial import build

@build('return x ** 2', replace=False)
def grow(x):
  return x + x

assert grow(3) == 6

If the alternative code does not compile, then the original body will not be replaced,

In [6]:
from loial import build

@build('return x to the power of 2')
def grow(x):
  return x + x

assert grow(3) == 6
print(f'Output: {grow(3)}')

ERROR:loial.builders.python_builder:Error compiling code, using defaut: invalid syntax (<string>, line 1)


Output: 6


# C builder example

These examples demonstrate how to compile C code to replace python functions.

This first example shows how to call a simple that take no arguments and returns an `int` value.

Note that this uses the `@cc_build` decorator, and unless decfined otherwise all values are assumed to be `int`.

Unlike with the python builder, the C code must include a function, which by default has the same name as the python function that is being replaced.

In [8]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun1() {
    return 10;
}
''')
def fun1():
    return 1

assert fun1() == 10

This next example passes in an int argument from python in to the C function and returns an `int` value

In [9]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun2(int a) {
    return a * 10;
}
''')
def fun2(a):
    return a

assert fun2(3) == 30

Multiple arguments are also supported.

In [10]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun3(int a, int b, int c) {
    return a + b + c;
}
''')
def fun3(a, b, c):
    return a*b*c

assert fun3(1, 2, 3) == 6

Calling the replaced python function also supports named arguments.

Here the arguments `d` & `e` are names and used in reverse order.

In [11]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun4(int a, int b, int c, int d, int e) {
    return ((a + b + c)*d)/e;
}
''')
def fun4(a, b, c, d, e):
    return (a*b*c+d)*e

assert fun4(1, 2, 3, e=2, d=10) == 30

Default python values are also supported when calling a replacement C function.

Here the default value for `e` is being used in the method call

In [12]:
from loial.builders.cc_builder import cc_build

@cc_build('''
int fun5(int a, int b, int c, int d, int e) {
    return ((a + b + c)*d)/e;
}
''')
def fun5(a, b, c, d, e=2):
    return (a*b*c+d)*e

assert fun5(1, 2, 3, d=10) == 30

If there is an error or the C code does not compile, then the default python implementation is used as a fallback.

However the compile failure is logged using standard python logging.

In [15]:
from loial.builders.cc_builder import cc_build

@cc_build('''
    junk
''')
def bad(a, b, c, d):
    return (a+b+c)*d

assert bad(1, 2, 3, 10) == 60

print(f'{"="*10}\nSuccessful result {bad(1, 2, 3, 10)}')

ERROR:loial.builders.cc_builder:Error compiling code: /root/.loial/lib__main__.bad_61337cf369816ff5d971d30baf628637.so.c:2:5: error: expected ‘=’, ‘,’, ‘;’, ‘asm’ or ‘__attribute__’ at end of input
    2 |     junk
      |     ^~~~
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/loial/builders/cc_builder.py", line 356, in cc_compile
    out = subprocess.run(
          ^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/subprocess.py", line 571, in run
    raise CalledProcessError(retcode, process.args,
subprocess.CalledProcessError: Command '['cc', '-I', '/tmp', '-fPIC', '-shared', '-o', '/root/.loial/lib__main__.bad_61337cf369816ff5d971d30baf628637.so', '/root/.loial/lib__main__.bad_61337cf369816ff5d971d30baf628637.so.c']' returned non-zero exit status 1.


Successful result 60


When building a replacement function, an extra configuraiton instance can be passed in.

This config can be used to auto delete the compiled shared object when the application exits.

When the function is replaced by the C code, the new function instance can be used to access the new callable replacement.

By default the `CC_Builder` will look through the search path to find a location to cache the compiled library.

In [19]:
from loial.builders.cc_builder import CC_Config

print(CC_Config().cache_search_path)

('/root/.loial', PosixPath('loial'))


In [22]:
!rm -rf /root/.loial

In [38]:
from loial.builders.cc_builder import cc_build, CC_Config

@cc_build('''
int delete_me() {
    return 10;
}
''', CC_Config(delete_on_exit=True))
def delete_me():
    return 1

assert delete_me(0) == 10

print(delete_me.callable.so_file)


/root/.loial/lib__main__.delete_me_e58d682f73d55cd5c53a8970829516b4.so


In [39]:
!ls /root/.loial*

Alternatively the global value can be set in the `CC_Config` class

In [40]:
from loial.builders.cc_builder import cc_build, CC_Config

CC_Config.delete_on_exit=True

@cc_build('''
int delete_me() {
    return 10;
}
''')
def delete_me():
    return 1

assert delete_me(0) == 10

print(delete_me.callable.so_file)

/root/.loial/lib__main__.delete_me_e58d682f73d55cd5c53a8970829516b4.so


In [41]:
!ls /root/.loial*

The default search path to find a cache directory can be configured in the `CC_Config` class.

In [42]:
CC_Config.cache_search_path = [f'./mycache']

@cc_build('''
int cached() {
    return 10;
}
''')
def cached():
    return 1

assert cached(0) == 10

print(cached.callable.so_file)

/content/mycache/lib__main__.cached_3acec0a97280c767692df798daf4a79e.so
